In [11]:
from IPython.display import display
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

# from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, average_precision_score
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import pickle

import lightgbm

import matplotlib.pyplot as plt

from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat')
gdf = source.read()[source.metadata['usecols']]

gdf = gdf.to_crs(epsg=4326)

In [ ]:
target = source.metadata['categories']['generic']
gdf[target] = gdf[target].astype('category')

In [ ]:
def load_array():
    all_data = []
    for i in range(1, 2):
        filepath = Path('data').joinpath(f'treesat_0{i}2019.npy')
        print(f'Loading {filepath}')
        with open(filepath, 'rb') as f:
            a = np.load(f)
            all_data.append(
                np.array(a.tolist()).reshape(len(a), -1)
            )
    
    return np.concatenate(all_data)

In [ ]:
data = load_array()
data = np.where(data > 0, data, data.mean())

In [ ]:
def train_lgbm(X_train, X_test, y_train, y_test, save_path):
    params = {
        # "boosting_type": "gbdt",
        # "objective": "regression",
        # "num_leaves": 31,
        # "learning_rate": 0.05,
        # "feature_fraction": 0.9,
        # "bagging_fraction": 0.8,
        # "bagging_freq": 5,
        "verbose": 0,
        # "device_type": "gpu"
    }

    lgb = lightgbm.LGBMClassifier(**params)
    
    clf = OneVsRestClassifier(lgb)
    print("Starting training...")
    clf.fit(X_train, y_train)

    with save_path.open(mode='wb') as f:
        pickle.dump(clf, f)
        
    # y_pred = clf.predict(X_test)
    # y_score = clf.predict_proba(X_test)
    
    return clf

In [ ]:
y = gdf[target].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
    data, y, test_size=0.1, random_state=42)

In [ ]:
%%time
save_path = Path('models').joinpath('lgbm.pickle')
lgbm = train_lgbm(X_train, X_test, y_train, y_test, save_path)